# Code for AMAS project 1
### *By Niels August Davidsen*

In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# URL of the webpage
url = 'https://www.nbi.dk/%7Ekoskinen/Teaching/AdvancedMethodsInAppliedStatistics2025/data/2014KenPomeroy.html'

# Send a GET request to the webpage
response = requests.get(url)
response.raise_for_status()  # Ensure the request was successful

# Parse the webpage content
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'ratings-table'})

# Extract table headers
headers = [th.get_text() for th in table.find('thead').find_all('th')]

# Extract table rows
rows = []
for row in table.find('tbody').find_all('tr'):
    cells = row.find_all(['th', 'td'])
    cells_text = [cell.get_text() for cell in cells]
    rows.append(cells_text)

# Create a DataFrame
df = pd.DataFrame(rows, columns=headers)

# Save to CSV
df.to_csv('kenpom_2014_ratings.csv', index=False)